# Cheatsheet pour le datasprint 2023

## Toflit18

## Récupérer tous les flux avec Marseille comme direction des fermes

In [ ]:
from csv import DictReader

flows = []
with open('../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['customs_region'] == 'Marseille':
            flows.append(row)
print(len(flows))

## Récupérer tous les flux d'import avec Marseille comme direction des fermes

In [ ]:
from csv import DictReader

flows = []
with open('../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['customs_region'] == 'Marseille' \
            and row['export_import'] == 'Imports' \
            :
            flows.append(row)
print(len(flows))

## Récupérer tous les flux d'import avec Marseille comme direction des fermes en 1789

In [ ]:
from csv import DictReader

flows = []
with open('../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['customs_region'] == 'Marseille' \
            and row['export_import'] == 'Imports' \
            and row['year'] == '1789'\
            :
            flows.append(row)
print(len(flows))

## Récupérer tous les flux d'import avec Marseille comme direction des fermes en 1789, seulement avec les sources qui contiennent produits et partenaires

In [ ]:
from csv import DictReader

flows = []
with open('../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['customs_region'] == 'Marseille' \
            and row['export_import'] == 'Imports' \
            and row['year'] == '1789' \
            and row['best_guess_region_prodxpart'] == '1' \
        :
            flows.append(row)
print(len(flows))

## Récupérer tous les flux d'export vers Marseille comme partenaire en 1789

In [ ]:
from csv import DictReader

flows = []
with open('../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['partner_simplification'] == 'Marseille' \
            and row['export_import'] == 'Exports' \
            and row['year'] == '1789' \
            :
            flows.append(row)
print(len(flows))

## Navigo

### Récupérer tous les pointcalls concernant Marseille

In [ ]:
from csv import DictReader

pointcalls = []
with open('../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['toponyme_fr'] == 'Marseille':
            pointcalls.append(row)
print(len(pointcalls))

# import json
# print(json.dumps(pointcalls[0], sort_keys=True, indent=4))

### Récupérer tous les pointcalls concernant Marseille en 1789

In [ ]:
from csv import DictReader

pointcalls = []
with open('../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['toponyme_fr'] == 'Marseille' \
           and row['date_fixed'].split('-')[0] == '1789':
            pointcalls.append(row)

print(len(pointcalls))
# import json
# print(json.dumps(pointcalls[0], sort_keys=True, indent=4))

### Récupérer tous les pointcalls concernant une **arrivée** à Marseille en 1789

In [ ]:
from csv import DictReader

pointcalls = []
with open('../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['toponyme_fr'] == 'Marseille' \
            and row['date_fixed'].split('-')[0] == '1789' \
            and row['pointcall_action'] == 'In'\
            :
            pointcalls.append(row)

print(len(pointcalls))
# import json
# print(json.dumps(pointcalls[0], sort_keys=True, indent=4))

### Récupérer la distribution des ports d'attache de tous les navires à Marseille en 1789

In [ ]:
from collections import Counter

homeports = Counter()
with open('../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['toponyme_fr'] == 'Marseille' \
            and row['date_fixed'].split('-')[0] == '1789' \
            and row['pointcall_action'] == 'In' \
            :
            homeport = row['homeport_toponyme_fr']
            homeports.update({homeport: 1})
            
homeports_as_array = [{"homeport": homeport if homeport != '' else 'sans port d\'attache déclaré', "count": count} for homeport, count in homeports.items()]
for p in sorted(homeports_as_array, key=lambda d : -d['count']):
    print(p['homeport'] + ' : ' + str(p['count']) + ' entrées')

### Récupérer toutes les cargaisons entrées à Marseille en 1789

In [ ]:
from csv import DictReader
import json
import re

def single_quote_double(single_quoted):
  cList=list(single_quoted)
  in_double=False;
  in_single=False;
  for i,c in enumerate(cList):
      if c=="'":
          if not in_double:
              in_single = not in_single
              cList[i]='"'
      elif c=='"':
          in_double = not in_double
  double_quoted = "".join(cList)    
  return double_quoted

def parse_all_cargos_field(str):
    all_cargos = []
    input_cargos_fixed = single_quote_double(str or '[]').replace("'", ' ')
    # the following line works to solve issues but sometimes makes python run into an infinite loop
    # python people help !
    # input_cargos_fixed = re.sub(r'\".*(\").*(\")\",', '', input_cargos_fixed, 2)
    
    try:
        all_cargos = json.loads(input_cargos_fixed)
    except ValueError:
        #print('===')
        #print('Error with : ')
        #print(str)
        #print(input_cargos_fixed)
        return []
    return all_cargos

cargos = []
with open('../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for i, row in enumerate(reader):
        if row['toponyme_fr'] == 'Marseille' \
           and row['date_fixed'].split('-')[0] == '1789':
            all_cargos = parse_all_cargos_field(row['all_cargos'])
            for cargo in all_cargos:
                if \
                        'cargo_item_action' in cargo \
                    and cargo['cargo_item_action'] == 'In'
                    \:
                    cargos.append(cargo)
print(len(cargos))
print(json.dumps(cargos[0], sort_keys=True, indent=4))

## Récupérer la liste des voyages déclarés à destination de Marseille

In [ ]:
from csv import DictReader

flows = []
with open('../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['destination_fr'] == 'Marseille' \
            :
            flows.append(dict(row))

print(len(flows))

## Récupérer la liste des voyages déclarés à destination de Marseille et arrivés en 1789

In [ ]:
from csv import DictReader

flows = []
with open('../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['destination_fr'] == 'Marseille' \
            and row['indate_fixed'].split('-')[0] == '1789' \
            :
            flows.append(dict(row))

print(len(flows))

## Récupérer la distribution des ports de départ pour les navires partis à Marseille en 1789

In [ ]:
from collections import Counter

departures = Counter()
with open('../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['destination_fr'] == 'Marseille' \
            and row['indate_fixed'].split('-')[0] == '1789' \
            :
            departure = row['departure_fr']
            departures.update({departure: 1})
            
departures_as_array = [{"departure": departure, "count": count} for departure, count in departures.items()]
for p in sorted(departures_as_array, key=lambda d : -d['count']):
    print(p['departure'] + ' : ' + str(p['count']) + ' voyages')

## Récupérer la liste des tonnages réalisée par Pierre Niccolo (valable pour tout autre csv en ligne)

In [ ]:
import requests
import csv

def get_online_csv(url):
  results = []
  with requests.Session() as s:
      download = s.get(url)
      decoded_content = download.content.decode('utf-8')
      reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
      for row in reader:
        results.append(dict(row))
  return results

TONNAGE_SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTYdeIwpzaVpY_KS91cXiHxb309iYBS4JN_1_hW-_oyeysuwcIpC2VJ5fWeZJl4tA/pub?output=csv'

tonnage_data = get_online_csv(TONNAGE_SPREADSHEET_URL)
print(tonnage_data)

## Récupérer la distribution des ports de départ pour les navires partis à Marseille en 1789, par tonnage cumulé selon les estimations de Pierre Niccolò

In [ ]:
from csv import DictReader
from collections import Counter
import requests
import csv

# 1. get online csv data
def get_online_csv(url):
  results = []
  with requests.Session() as s:
      download = s.get(url)
      decoded_content = download.content.decode('utf-8')
      reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
      for row in reader:
        results.append(dict(row))
  return results

TONNAGE_SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTYdeIwpzaVpY_KS91cXiHxb309iYBS4JN_1_hW-_oyeysuwcIpC2VJ5fWeZJl4tA/pub?output=csv'

tonnage_data = get_online_csv(TONNAGE_SPREADSHEET_URL)

# 2. build a shipclass->tx dict
tonnage_estimates = {}
for l in tonnage_data:
    estimation = l['tonnage_estime_en_tx'] or 0
    if l['tonnage_estime_en_tx'] == 'No data':
        estimation = 0
    else :
        estimation = int(estimation)
    
    ship_class = l['ship_class']
    
    tonnage_estimates[ship_class] = estimation

# 1. count departures aggregating by tonnage
departures = Counter()
with open('../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        # filter relevant flows
        if \
                row['destination_fr'] == 'Marseille' \
            and row['indate_fixed'].split('-')[0] == '1789' \
            :
            departure = row['departure_fr']
            ship_class = row['ship_class_standardized']
            tonnage = 0
            # check ship_class is covered by our estimates
            if ship_class in tonnage_estimates:
                tonnage = tonnage_estimates[ship_class]
            elif ship_class != '':
                # note : should we use the built-in tonnage instead of doing nothing here?
                print('not in estimates : ' + ship_class)
                
            # update counter
            departures.update({departure: tonnage})
            
departures_as_array = [{"departure": departure, "count": count} for departure, count in departures.items()]
for p in sorted(departures_as_array, key=lambda d : -d['count']):
    print(p['departure'] + ' : ' + str(p['count']) + ' tonneaux cumulés')